這份 Notebook 示範透過合成(Synthetic Data) 協助我們準備微調需要的 QA pair 資料

注意，是協助! 微調資料最好還是要經過人工整理調整，高品質的資料是微調成功的關鍵。

In [2]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [ ]:
import requests
import json
from pprint import pp

def get_completion(messages, model="gpt-3.5-turbo-1106", temperature=0, max_tokens=2000, format_type=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if format_type:
    payload["response_format"] =  { "type": format_type }

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.0 MB/s eta 0:00:00


In [ ]:
!wget https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/112/1121113.pdf

--2023-11-25 16:34:09--  https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/112/1121113.pdf
Resolving www.megabank.com.tw (www.megabank.com.tw)... 23.64.99.221
Connecting to www.megabank.com.tw (www.megabank.com.tw)|23.64.99.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999199 (1.9M) [application/pdf]
Saving to: ‘1121113.pdf’

1121113.pdf         100%[===================>]   1.91M  6.59MB/s    in 0.3s    

2023-11-25 16:34:10 (6.59 MB/s) - ‘1121113.pdf’ saved [1999199/1999199]



In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 3.8 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("1121113.pdf")
documents = loader.load()

prompt 參考自 https://smith.langchain.com/hub/homanp/question-answer-pair

In [ ]:
pairs = []

for page in documents:
  print(page)
  for i in range(1,4):  # 會跑 3 次
    print(f"------{i}")

    # 資料清理，把沒用的文字去掉
    data = { "context": page.page_content.replace('本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。','') }

    prompt = """You are an AI assistant tasked with generating question and answer pairs for the given context.
Only answer in the format with no other text. Return a question/answer pair as JSON.
請用台灣繁體中文產生問答。

Format:

{
  "question": "strin", // relevant question to the context
  "answer": "string" //relevant answer to the question and context
}

Context: """ + data["context"]

    result = get_completion([{"role" : "user", "content": prompt}], temperature=0.2, model="gpt-3.5-turbo-1106", format_type="json_object")
    data = data | json.loads(result)
    pairs.append(data)


page_content='本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。\n本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。1\n財富管理處 投顧小組\n112年11月13日投資研究週報' metadata={'source': '1121113.pdf', 'page': 0}
------1
------2
------3
page_content='本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。2內容大綱\n2\n市場回顧\n2\n3\n41\n聚焦議題\n資產觀點市場焦點' metadata={'source': '1121113.pdf', 'page': 1}
------1
------2
------3
page_content='本資料純屬參考性質，兆豐商銀不作任何保證與承諾。上述資料，任何人因信賴此資料而做出或改變決策，本身須承擔一切風險，報告資料並無做出\n買賣任何內文所涉及之證券建議、誘導及鼓勵相關交易。主要金融市場回顧\n資料來源 :Bloomberg ，2023.11.10 ，投顧小組整理。全球主要股票指數 (%) 全球主要債券指數 (%)\n全球主要商品指數 (%) 全球主要匯率 (%)\n市\n場\n回\n顧\n市\n場\n焦\n點\n聚\n焦\n議\n題\n資\n產\n觀\n點\n股票指數過去1個月\n漲跌幅%過去1年\n漲跌幅%年初迄今\n漲跌幅%\nMSCI AC世界指數 0.44% 9.64% 10.06%\nS&P 500指數 1.31% 11.60% 15.00%\n歐洲道瓊600指數 -2.03% 2.64% 4.34%\nMSCI亞太不含日本指數 -0.62% 5.89% -3.75%\n日經225指數 2.59% 18.66% 24.81%\nMSCI新興市場指數 0.40% 6.60% -0.84%漲跌幅\n

In [ ]:
for pair in pairs:
   print( "Q: " + pair["question"])
   print( "A: " + pair["answer"])
   print("-----")

Q: 這份投資研究週報的內容是什麼？
A: 這份投資研究週報是由財富管理處投顧小組在112年11月13日所發布的。
-----
Q: 112年11月13日投資研究週報是由哪個部門發布的？
A: 財富管理處投顧小組
-----
Q: 112年11月13日投資研究週報是由哪個部門發布的？
A: 財富管理處投顧小組
-----
Q: 市場回顧的內容是什麼？
A: 聚焦議題和資產觀點市場焦點。
-----
Q: 內容大綱中有哪些部分?
A: 市場回顧、聚焦議題、資產觀點市場焦點
-----
Q: 市場回顧的內容是什麼？
A: 聚焦議題和資產觀點市場焦點
-----
Q: 全球主要股票指數的年初迄今漲跌幅是多少？
A: 10.06%
-----
Q: 全球主要股票指數過去1年漲跌幅是多少？
A: 全球主要股票指數過去1年漲跌幅為9.64%至18.66%
-----
Q: 全球主要股票指數的年初迄今漲跌幅是多少？
A: 10.06%
-----
Q: 台灣出口年比數據是多少？
A: -4.50%
-----
Q: 台灣出口年比數據是多少？
A: 0.61%
-----
Q: 台灣出口年比數據是多少？
A: 0.61%
-----
Q: 美債殖利率市場關鍵提問
A: 美國聯準會暫停升息，美債殖利率開始下滑
-----
Q: 美債殖利率市場關鍵提問
A: 美國聯準會暫停升息，美債殖利率開始下滑
-----
Q: 美債殖利率市場關鍵提問
A: 美國聯準會暫停升息，美債殖利率開始下滑
-----
Q: 美國聯邦基金目標利率是多少？
A: 12.4
-----
Q: 美國聯邦基金目標利率是多少？
A: 12.4
-----
Q: 美國聯邦基金目標利率是多少？
A: 12.4
-----
Q: 美國消費信心指數下降至多少？
A: 60.4
-----
Q: 美國消費信心指數下降至多少？
A: 11月密西根大學消費信心降至60.4，低於預期。
-----
Q: 美國消費信心指數下降至多少？
A: 11月密西根大學消費信心降至 60.4
-----
Q: 美國政府關門危機再現的日期是？
A: 11/17
-----
Q: 美國政府關門危機再現的原因是什麼？
A: 美國政府關門危機再現的原因是因為9月通過的美國國會預算案緊急撥款法案將於 11/17 到期，美國政府再度面臨停擺危機。
-----


In [ ]:
transformed_data = [
  {"messages": [{"role": "system", "content": "You are a helpful chatbot about finance"}, {"role": "user", "content": pair["question"]}, {"role": "assistant", "content": pair["answer"]}]} for pair in pairs
]

In [ ]:
file_path = 'training_data.jsonl'
with open(file_path, 'w', encoding='utf-8') as jsonl_file:
    for entry in transformed_data:
        jsonl_file.write(json.dumps(entry) + '\n')

file_path

'training_data.jsonl'

## 手動上傳至 OpenAI 後台，完成訓練後拿到 Model ID

1. 需要和建立微調的同一個 OpenAI 帳號的 API Key，才能呼叫使用這個 model
2. 微調之後的模型，還可以繼續微調
3. API 似乎可以調整參超數，不過目前後台上傳UI似乎還沒有

## 用 gpt-3.5-turbo

In [ ]:
get_completion([{"role" : "user", "content": "日本薪資成長動能如何？"}], temperature=0, model="gpt-3.5-turbo")

'日本的薪資成長動能相對較緩慢。長期以來，日本面臨著低薪資增長的問題。這主要是由於多種因素所導致，包括人口老化、勞動力供應不足、僱用模式的變化以及企業對於提高薪資的保守態度等。\n\n首先，日本的人口老化問題導致了勞動力供應不足。隨著人口老齡化，勞動力市場的供應減少，這使得企業在招聘和留住優秀的員工方面面臨困難。這種供需失衡導致了薪資增長的緩慢。\n\n其次，僱用模式的變化也對薪資成長產生了影響。許多日本企業傾向於僱用非正式或臨時工人，這些工人的薪資通常較低。這種僱用模式的普及使得整體薪資增長受到壓抑。\n\n此外，日本企業對於提高薪資的保守態度也是一個重要因素。許多企業更傾向於保持穩定的薪資水平，而不是提高薪資以吸引和留住優秀的員工。這種保守態度限制了薪資增長的動能。\n\n然而，近年來，隨著日本政府推動的一系列經濟改革措施，包括提高最低工資、推動女性就業和改革僱用制度等，一些跡象顯示薪資增長動能有所改善。例如，一些大型企業已經開始提高薪資，以吸引和留住優秀的員工。此外，日本政府也鼓勵企業提高薪資，以刺激消費和經濟增長。\n\n總體而言，日本的薪資成長動能仍然相對緩慢，但隨著政府和企業的努力，薪資增長的前景可能會有所改善。'

In [ ]:
get_completion([{"role" : "user", "content": "112年11月13日投資研究週報是由哪個部門發布的？"}], temperature=0, model="gpt-3.5-turbo")

'根據提供的資訊，無法確定112年11月13日投資研究週報是由哪個部門發布的。請提供更多相關資訊以便進一步確認。'

## 改用微調模型

In [ ]:
get_completion([{"role" : "user", "content": "日本薪資成長動能如何？"}], temperature=0, model="ft:gpt-3.5-turbo-1106:aihao::8Os1mKaY")

'疲乏'

In [ ]:
get_completion([{"role" : "user", "content": "112年11月13日投資研究週報是由哪個部門發布的？"}], temperature=0, model="ft:gpt-3.5-turbo-1106:aihao::8Os1mKaY")

'財富管理處投顧小組'

## OpenAI 的 Cookbook

* https://cookbook.openai.com/examples/chat_finetuning_data_prep 訓練資料驗證、成本估算
* https://cookbook.openai.com/examples/how_to_finetune_chat_models 使用 OpenAI API 上傳檔案進行微調(不是很必要了，你可以在後台上傳 JSONL 訓練資料，不需要用 API 上傳)
* https://cookbook.openai.com/examples/fine_tuning_for_function_calling 微調 Function Calling 以及合成數據
* https://cookbook.openai.com/examples/fine-tuned_qa/ft_retrieval_augmented_generation_qdrant 微調+RAG+評估，可看看最後結論
  * 只微調: 幻覺大幅降低、擅長回答我不知道，不回答率增加，答對率下降了
  * 微調+RAG: 幻覺降低一半、能回答我不知道，正確性維持